In [39]:
#Obter capacidade
Capacidade = EV['Total Energy (kWh)'].max()
print(Capacidade)

#Obter potencia média dos carregadores
a = (EV.groupby('Charging Station')['Duration (min)'].mean())
b = (EV.groupby('Charging Station')['Total Energy (kWh)'].mean())
print(b/(a/60))


55.51
Charging Station
CAP_SON_MAI_00001     5.064675
CAP_SON_MAI_00002    63.506413
CAP_SON_MAI_00003     5.547814
CAP_SON_MAI_00004    10.853716
CAP_SON_MAI_00010     6.838004
dtype: float64


In [38]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

EV = pd.read_csv("EVIO_history_01-06-2022_31-05-2023.csv", sep=';')

# Convert strings to float
EV['Duration (min)'] = EV['Duration (min)'].str.replace(',', '.').astype(float)
EV['Cost incl. IVA'] = EV['Cost incl. IVA'].str.replace(',', '.').astype(float)
EV['Total Energy (kWh)'] = EV['Total Energy (kWh)'].str.replace(',', '.').astype(float)
EV['Start date'] = pd.to_datetime(EV['Start date'], format='%m/%d/%Y | %H:%M')
EV['Stop date'] = pd.to_datetime(EV['Stop date'], format='%m/%d/%Y | %H:%M')

cost = EV['Cost incl. IVA']
energy = EV['Total Energy (kWh)']
duration = EV['Duration (min)']
EV['Day of Week'] = EV['Start date'].dt.day_name()

# Criar colunas separadas para hora e minuto
EV['start_hour'] = EV['Start date'].dt.hour
EV['start_minute'] = EV['Start date'].dt.minute

# Calcular a média da duração para cada dia da semana
avg_duration = EV.groupby('Day of Week')['Duration (min)'].mean()

# Agrupar por dia da semana e hora/minuto
grouped = EV.groupby(['Day of Week', 'start_hour', 'start_minute'])

# Contar as ocorrências de cada hora/minuto para cada dia da semana
counts = grouped.size().reset_index(name='count')

# Encontrar o máximo para cada dia da semana
idx = counts.groupby('Day of Week')['count'].idxmax()
result = counts.loc[idx].reset_index()

result['Start Time'] = result['start_hour'].astype(str).str.zfill(2) + ':' + result['start_minute'].astype(str).str.zfill(2)

#result['New Column'] = result.apply(lambda row: EV.loc[(EV['Day of Week'] == row['Day of Week']) & (EV['start_hour'] == row['start_hour']) & (EV['start_minute'] == row['start_minute']), 'Start date'].iloc[0] + pd.Timedelta(minutes=avg_duration[row['Day of Week']]), axis=1)
result['Endtime'] = result.apply(lambda row: (EV.loc[(EV['Day of Week'] == row['Day of Week']) & (EV['start_hour'] == row['start_hour']) & (EV['start_minute'] == row['start_minute']), 'Start date'].iloc[0] + pd.Timedelta(minutes=avg_duration[row['Day of Week']])).strftime('%H:%M'), axis=1)

# Adicionar coluna da duração média
result['Average Duration'] = result.apply(lambda row: avg_duration[row['Day of Week']], axis=1)

# Exibir os resultados
print(result[['Day of Week', 'Start Time', 'Endtime', 'Average Duration' ]])


# Ordenar o DataFrame por 'Day of Week', 'start_hour', 'start_minute' para garantir a ordem correta no gráfico
result = result.sort_values(['Day of Week', 'start_hour', 'start_minute'])

  Day of Week Start Time Endtime  Average Duration
0      Friday      08:27   11:06        159.994059
1      Monday      08:23   10:54        151.898059
2    Saturday      08:24   08:43         19.922333
3      Sunday      09:10   09:30         20.167000
4    Thursday      08:15   10:07        112.656286
5     Tuesday      08:07   11:07        180.869259
6   Wednesday      13:11   16:22        191.112968


In [6]:
# Criar uma nova coluna 'Datetime' que combina 'Day of Week', 'Start Time' e 'End Time' para plotagem
result['Datetime'] = result['Day of Week'] + ' ' + result['Start Time'] + '-' + result['Endtime']

# Criar uma coluna 'Timestamp' para ordenar corretamente no eixo x
result['Timestamp'] = pd.to_datetime(result['Datetime'], format='%A %H:%M-%H:%M')

# Plotar o gráfico
plt.figure(figsize=(12, 6))
plt.plot(result['Timestamp'], result.index, marker='o')
plt.title('Início e Fim do Carregamento por Dia da Semana')
plt.xlabel('Horário')
plt.ylabel('Dia da Semana')
plt.grid(True)
plt.show()

error: redefinition of group name 'H' as group 4; was group 2 at position 113